Option chains
=======

In [3]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=23)

<IB connected to 127.0.0.1:7497 clientId=23>

In [7]:
a = ib.positions()
option_positions = [x for x in a if x.contract.secType== 'OPT']
option_positions_dict = {}
for option_position in option_positions:
    option_positions_dict[option_position.contract] = option_position.position


In [14]:
for (contract, position) in option_positions_dict.items():
    print(contract.symbol,contract.lastTradeDateOrContractMonth, 
                          contract.strike,contract.right,
                          'SMART', contract.tradingClass)
    contract_new = Option(contract.symbol,contract.lastTradeDateOrContractMonth, 
                          contract.strike,contract.right,
                          'SMART', tradingClass = contract.tradingClass)
#     contract_new = Option('AMC', '20210618', '145', 'C', 'SMART', tradingClass='AMC')
#     ib.qualifyContracts(spx)
    ib.qualifyContracts(contract_new)
    
    [ticker] = ib.reqTickers(contract_new)
    print(ticker.modelGreeks.delta)

AMC 20210618 73.0 C SMART AMC
0.45727501449317826
AMC 20210618 30.0 P SMART AMC
-0.15338782289153807
AMC 20210716 60.0 C SMART AMC
0.6349186852540254
AMC 20210618 145.0 C SMART AMC
0.21123109540346766
ROKU 20210917 260.0 P SMART ROKU


AttributeError: 'NoneType' object has no attribute 'delta'

Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.


Suppose we want to find the options on the SPX, with the following conditions:

* Use the next three monthly expiries;
* Use strike prices within +- 20 dollar of the current SPX value;
* Use strike prices that are a multitude of 5 dollar.

To get the current market value, first create a contract for the underlyer (the S&P 500 index):

In [4]:
spx = Option('AMC', '20210618', '145', 'C', 'SMART', tradingClass='AMC')
ib.qualifyContracts(spx)

[Option(conId=495048824, symbol='AMC', lastTradeDateOrContractMonth='20210618', strike=145.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='AMC   210618C00145000', tradingClass='AMC')]

To avoid issues with market data permissions, we'll use delayed data:

In [5]:
ib.reqMarketDataType(4)

Then get the ticker. Requesting a ticker can take up to 11 seconds.

In [6]:
[ticker] = ib.reqTickers(spx)
ticker

Ticker(contract=Option(conId=495048824, symbol='AMC', lastTradeDateOrContractMonth='20210618', strike=145.0, right='C', multiplier='100', exchange='SMART', currency='USD', localSymbol='AMC   210618C00145000', tradingClass='AMC'), time=datetime.datetime(2021, 6, 6, 19, 40, 11, 347770, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=7.62, halted=0.0, bidGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=4.75, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=None), askGreeks=OptionComputation(impliedVol=None, delta=None, optPrice=4.800000190734863, pvDividend=0.0, gamma=None, vega=None, theta=None, undPrice=None), lastGreeks=OptionComputation(impliedVol=4.920917874649643, delta=0.2124163512585574, optPrice=4.760000228881836, pvDividend=0.0, gamma=0.006777892242864399, vega=0.023433621654397552, theta=-0.5178497739815338, undPrice=None), modelGreeks=OptionComputation(impliedVol=4.909418598919583, delta=0.21134981274024725, optPrice=3.54

Take the current market value of the ticker:

In [5]:
spxValue = ticker.marketPrice()
spxValue

4192.85

The following request fetches a list of option chains:

In [6]:
chains = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)

util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,SMART,416904,SPX,100,"[20210617, 20210715, 20210819, 20210916, 20211...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
1,CBOE,416904,SPXW,100,"[20210607, 20210609, 20210611, 20210614, 20210...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
2,SMART,416904,SPXW,100,"[20210607, 20210609, 20210611, 20210614, 20210...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."
3,CBOE,416904,SPX,100,"[20210617, 20210715, 20210819, 20210916, 20211...","[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700..."


These are four option chains that differ in ``exchange`` and ``tradingClass``. The latter is 'SPX' for the monthly and  'SPXW' for the weekly options. Note that the weekly expiries are disjoint from the monthly ones, so when interested in the weekly options the monthly options can be added as well.

In this case we're only interested in the monthly options trading on SMART:

In [7]:
chain = next(c for c in chains if c.tradingClass == 'SPX' and c.exchange == 'SMART')
chain

OptionChain(exchange='SMART', underlyingConId='416904', tradingClass='SPX', multiplier='100', expirations=['20210617', '20210715', '20210819', '20210916', '20211014', '20211118', '20211216', '20220120', '20220217', '20220317', '20220413', '20220519', '20220616', '20221215', '20231214'], strikes=[100.0, 200.0, 300.0, 400.0, 500.0, 600.0, 700.0, 800.0, 900.0, 1000.0, 1100.0, 1150.0, 1200.0, 1225.0, 1250.0, 1275.0, 1300.0, 1325.0, 1350.0, 1375.0, 1400.0, 1425.0, 1450.0, 1475.0, 1500.0, 1525.0, 1550.0, 1575.0, 1600.0, 1625.0, 1650.0, 1675.0, 1700.0, 1725.0, 1750.0, 1775.0, 1800.0, 1825.0, 1850.0, 1875.0, 1900.0, 1925.0, 1950.0, 1975.0, 2000.0, 2025.0, 2050.0, 2075.0, 2100.0, 2125.0, 2150.0, 2175.0, 2200.0, 2225.0, 2250.0, 2275.0, 2300.0, 2325.0, 2350.0, 2375.0, 2400.0, 2425.0, 2450.0, 2475.0, 2500.0, 2525.0, 2550.0, 2575.0, 2590.0, 2600.0, 2610.0, 2620.0, 2625.0, 2630.0, 2640.0, 2650.0, 2660.0, 2670.0, 2675.0, 2680.0, 2690.0, 2700.0, 2710.0, 2720.0, 2725.0, 2730.0, 2740.0, 2750.0, 2760.0, 

What we have here is the full matrix of expirations x strikes. From this we can build all the option contracts that meet our conditions:

In [8]:
strikes = [strike for strike in chain.strikes
        if strike % 5 == 0
        and spxValue - 20 < strike < spxValue + 20]
expirations = sorted(exp for exp in chain.expirations)[:3]
rights = ['P', 'C']

contracts = [Option('SPX', expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights
        for expiration in expirations
        for strike in strikes]

contracts = ib.qualifyContracts(*contracts)
len(contracts)

Error 200, reqId 18: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20210715', strike=4185.0, right='P', exchange='SMART', tradingClass='SPX')
Error 200, reqId 20: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20210715', strike=4195.0, right='P', exchange='SMART', tradingClass='SPX')
Error 200, reqId 22: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20210715', strike=4205.0, right='P', exchange='SMART', tradingClass='SPX')
Error 200, reqId 26: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20210819', strike=4185.0, right='P', exchange='SMART', tradingClass='SPX')
Error 200, reqId 28: No security definition has been found for the request, contract: Option(symbol='SPX', lastTradeDateOrContractMonth='20210819', stri

36

In [9]:
contracts[0]

Option(conId=458065953, symbol='SPX', lastTradeDateOrContractMonth='20210617', strike=4175.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   210618P04175000', tradingClass='SPX')

Now to get the market data for all options in one go:

In [10]:
tickers = ib.reqTickers(*contracts)

tickers[0]

Ticker(contract=Option(conId=458065953, symbol='SPX', lastTradeDateOrContractMonth='20210617', strike=4175.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   210618P04175000', tradingClass='SPX'), time=datetime.datetime(2021, 6, 6, 17, 12, 49, 984650, tzinfo=datetime.timezone.utc), bid=-1.0, bidSize=0, ask=-1.0, askSize=0, close=37.85)

In [13]:
a = tickers[0]
a.bidSize

0

In [16]:
a = contracts[0]
a

Option(conId=458065953, symbol='SPX', lastTradeDateOrContractMonth='20210617', strike=4175.0, right='P', multiplier='100', exchange='SMART', currency='USD', localSymbol='SPX   210618P04175000', tradingClass='SPX')

The option greeks are available from the ``modelGreeks`` attribute, and if there is a bid, ask resp. last price available also from ``bidGreeks``, ``askGreeks`` and ``lastGreeks``. For streaming ticks the greek values will be kept up to date to the current market situation.

In [11]:
ib.disconnect()